# Ατομική Εργασία Στην Επιχειρησιακή Έρευνα

###  Ονοματεπώνυμο: Λιόλιος Αναστάσιος 
###  ΑΕΜ: 9171
###  Email: lioliosaa@ece.auth.gr

### Ερώτημα 1 - Σχέδιο Σωτηρία 1
Στο πρώτο ερώτημα της ατομικής εργασίας το πρόβλημα που πρέπει να μοντελοποιήσουμε είναι η ελαχιστοποίηση των διαθέσιμων 8 οχημάτων για τη μεταφορά των σημαντικών εγγράφων απο την πόλη Α στις 14 διαθέσιμες πόλεις B(1,..,14), όπου η απόσταση που καλύπτει το κάθε όχημα είναι το πολύ 400 χιλιόμετρα και δεν επιτέπεται ανεφοδιασμός των οχημάτων. Επίσης, από την εκφώνηση γνωρίζουμε ότι κάθε φορτηγό μπορεί να μεταφέρει όλα τα αρχεία, ενώ πρέπει να φτάσει τουλάχιστον ένα αρχείο σε κάθε πόλη.

Όπως μπορούμε να καταλάβουμε, το πρόβλημα μας αντιστοιχεί σε μια ειδική περίπτωση του γνωστού προβλήματος Vehicle Routing Problem. Επομένως, η διαδικασία στη μοντελοποίηση που ακολουθήσαμε είναι παρόμοια με την μοντελοποίηση του δύσκολου αυτού προβλήματος.

Παρακάτω, λοιπόν παρουσιάζουμε τον κώδικα για το πρώτο ερώτημα ο οποίος είναι τεκμηριωμένος με σχόλια για την κατανόηση του.

In [17]:
import numpy as np
import matplotlib.pyplot as plt


distances = np.array([0, 180, 240, 85, 285, 205, 235, 255, 155, 120, 230, 340, 220, 160, 240,
                 180, 0, 255, 150, 125, 100, 175, 235, 25, 65, 95, 210, 55, 135, 215,
                 240, 255, 0, 160, 235, 160, 105, 45, 245, 255, 195, 225, 310, 115, 50,
                 85, 150, 160, 0, 215, 125, 150 , 170, 130, 110, 160, 260, 200, 75, 150,
                 285, 125, 235, 215, 0, 90, 130, 195, 150, 190 ,50, 85, 155, 155, 185,
                 205, 100, 160, 125, 90, 0, 70, 135, 110, 135, 40, 135, 155, 65, 120,
                 235, 175, 105, 150, 130, 70, 0, 70, 175, 200, 95, 125, 225, 70, 55,
                 255, 235, 45, 170, 195, 135, 70, 0 , 235, 250, 165, 180, 290, 110, 20,
                 155, 25, 245, 130, 150, 110, 175, 235, 0, 40, 110, 225, 70, 130, 215,
                 120, 65, 255, 110, 190, 135, 200, 250, 40, 0, 145, 265, 100, 135, 230,
                 230, 95, 195, 160, 50, 40, 95, 165, 110, 145, 0, 115, 135, 105, 150,
                 340, 210, 225, 260, 85, 135, 125, 180, 225, 265, 115, 0, 240, 185, 175,
                 220, 55, 310, 200, 155, 155, 225, 290, 70, 100, 135, 240, 0, 190, 270,
                 160, 135, 115, 75, 155, 65, 70, 110, 130, 135, 105, 185, 190, 0, 90,
                 240, 215, 50, 150, 185, 120, 55, 20, 215, 230, 150, 175, 270, 90, 0])

distances = distances.reshape((15, 15))

# The vehicles' limit on distance
D = 400

# The number of B cities and warehouse A
n = 15
node = [i for i in range(1,n+1)]

# The number of vehicles' tour that we use to find the minimum vehicles
vehicle_tour = [1, 2, 3]

# Create a dictionary with distances. We consider A = 1 and B = (2, ..., 15)
dist = {(i, j):distances[i - 1][j - 1] for i in node for j in node}

from gurobipy import *


# Create a function for subtour elimination with lazy constraints
def subtourelim(model, where):
    if where == GRB.callback.MIPSOL:
        selected = []
        # for i in range(n):
        sol = model.cbGetSolution(model._y)
        for e in vehicle_tour:
            selected = tuplelist((i, j) for i, j, k in model._y.keys() if sol[i, j, k] > 0.5 if k == e)
            adjList = [[] for i in range(n + 1)]
            for i, j in selected:
                adjList[i].append(j)
            # find the shortest cycle in the selected edge list
            components = subtour(adjList)
            #print(components)
            count = 0
            if len(components) > 1:
                # add a subtour elimination constraint
                for component in components:
                    if len(component) >= 2:
                        count += 1
                if count > 1:
                    for component in components:
                        if (len(component) >= 2):
                            #print('Add lazy constraint for component: {}'.format(component))
                            m.cbLazy(quicksum(t[i, j] for i in component for j in component if i != j ) <= len(component) - 1)
                            

# Find the subtours in order to use them in the above function
def subtour(adjList):
    discover = [0 for i in range(n + 1)]
    components = []
    for i in range(n + 1):
        component = []
        queue = []
        if discover[i] == 0:
            discover[i] = 1
            component.append(i)
            queue.append(i)
            while queue:
                v = queue.pop(0)
                for u in adjList[v]:
                    if discover[u] == 0:
                        discover[u] = 1
                        component.append(u)
                        queue.append(u)
            components.append(component)
    return components

# Initialize a Gurobi Model
m = Model()

# From here starts the modelling of the problem and creating the right constraints

# x[i, j] = 1 if node i is assigned to tour j
x = {}
for i in node:
    for j in vehicle_tour:
        x[i, j] = m.addVar(vtype = GRB.BINARY, name = 'x'+ str(i) + str(j))
        
# y[i, j, k] = 1 if arc(i,j) is assigned to tour k
y = {}
for i in node:
    for j in node:
        for k in vehicle_tour:
            y[i, j, k] = m.addVar(vtype = GRB.BINARY, name = 'y' + str(i) + str(j) + str(k))
            
t = {}
for i in node:
    for j in node:
        t[i, j] = m.addVar(vtype = GRB.BINARY, name = 't' + str(i) + str(j))
        

z1={}

z1=m.addVar(vtype=GRB.CONTINUOUS,name='z1',obj=1)

z2={}

z2=m.addVar(vtype=GRB.CONTINUOUS,name='z2',obj=1)

z3={}

z3=m.addVar(vtype=GRB.CONTINUOUS,name='z3',obj=1)


m.update()
m.modelsense = GRB.MINIMIZE

# Minimize the distance tour of every vehicle
m.addConstr(quicksum(y[i,j,k]*dist[i,j] for i in node for j in node for k in vehicle_tour) - quicksum(dist[i, 1] * y[i, 1, 1] for i in node)  <= z1)
m.addConstr(quicksum(y[i,j,k]*dist[i,j] for i in node for j in node for k in vehicle_tour) - quicksum(dist[i, 1] * y[i, 1, 2] for i in node)  <= z2)
m.addConstr(quicksum(y[i,j,k]*dist[i,j] for i in node for j in node for k in vehicle_tour) - quicksum(dist[i, 1] * y[i, 1, 3] for i in node)  <= z3)

# Every node should be assigned to at least one tour
for i in range(2, len(node) + 1):
    m.addConstr(quicksum(x[i, j] for j in vehicle_tour) == 1)
    
# Total distance of a particular trip shall be less than D
for k in vehicle_tour:
    m.addConstr(quicksum(dist[i, j] * y[i, j, k] for i in node for j in node) - quicksum(dist[i, j] * y[i, j, k] for i in node for j in range(1, 2)) <= D)

# Every node should have exactly one edge leaving
for i in range(2, len(node) + 1):
    m.addConstr(quicksum(y[i, j, k] for j in node for k in vehicle_tour) == 1)
    
# Every node should have exactly one edge entering
for i in range(2, len(node) + 1):
    m.addConstr(quicksum(y[j, i, k] for j in node for k in vehicle_tour) == 1)

# (y[i, i, k] = 0
for i in node:
    m.addConstr(quicksum(y[i, i, k] for k in vehicle_tour) == 0)
    
# for every node i the previous and next node shall be in same route
for i in range(2, len(node) + 1):
    for k in vehicle_tour:
        m.addConstr(quicksum(y[i, j, k] for j in node) == quicksum(y[j, i, k] for j in node))

# linking x and y
for i in range(2,len(node) + 1):
    for k in vehicle_tour:
        m.addConstr(x[i, k] == quicksum(y[i, j, k] for j in node))
        
# linking y and t
for i in node:
    for j in node:
        m.addConstr(quicksum(y[i, j, k] for k in vehicle_tour) == t[i, j])

# node 1 should appear in all route
for k in vehicle_tour:
    m.addConstr(x[1, k] == 1)

# number of edges entering is equal to number of edges leaving
m.addConstr(quicksum(y[1, i, k] for i in node for k in vehicle_tour) == quicksum(y[i, 1, k] for i in node for k in vehicle_tour))

# atleast one edge outgoing from node 1 in every tour
for k in vehicle_tour:
    m.addConstr(quicksum(y[1, j, k] for j in node) == 1)
    
# atleast one edge incoming to node 1 in every tour
for k in vehicle_tour:
    m.addConstr(quicksum(y[j, 1, k] for j in node) == 1)

# Prints the binary variables x,y,t and the optimal solution
def print_sol():
    if m.status == GRB.status.OPTIMAL:
        for i in node:
            for k in vehicle_tour:
                if x[i, k].x > 0.5:
                    print('x[%d,%d]=1'%(i, k))

        for i in node:
            for j in node:
                for k in vehicle_tour:
                    if y[i, j, k].x > 0.5:
                        print('y[%d,%d,%d]=1'%(i, j, k))
        for i in node:
            for j in node:
                if t[i, j].x > 0.5:
                    print('t[%d,%d]=1'%(i, j))
                    
        vehicle_tour1 = [1]
        vehicle1_distance = 0
        a = 1
        counter = 0
        while counter < 1:
            for j in node:
                if y[a, j, 1].x > 0.5:
                    if j != 1:
                        vehicle_tour1.append(j)
                        vehicle1_distance += dist[a, j]
                    a = j
                    if a == 1:
                        counter += 1
                        break
                        
        vehicle_tour2 = [1]
        vehicle2_distance = 0
        a = 1
        counter = 0
        while counter < 1:
            for j in node:
                if y[a, j, 2].x > 0.5:
                    if j != 1:
                        vehicle_tour2.append(j)
                        vehicle2_distance += dist[a, j]
                    a = j
                    if a == 1:
                        counter += 1
                        break
                        
        vehicle_tour3 = [1]
        vehicle3_distance = 0
        a = 1
        counter = 0
        while counter < 1:
            for j in node:
                if y[a, j, 3].x > 0.5:
                    if j != 1:
                        vehicle_tour3.append(j)
                        vehicle3_distance += dist[a, j]
                    a = j
                    if a == 1:
                        counter += 1
                        break
        
        for i in range(len(vehicle_tour1)):
            vehicle_tour1[i] = vehicle_tour1[i] - 1
        print("Optimal Solution Found!!!")
        print("Vehicle tour 1: " + str(vehicle_tour1))
        print("Distance of vehicle tour 1: " + str(vehicle1_distance))
        
        for i in range(len(vehicle_tour2)):
            vehicle_tour2[i] = vehicle_tour2[i] - 1
        print("Vehicle tour 2: " + str(vehicle_tour2))
        print("Distance of vehicle tour 2: " + str(vehicle2_distance))
        
        for i in range(len(vehicle_tour3)):
            vehicle_tour3[i] = vehicle_tour3[i] - 1
        print("Vehicle tour 3: " + str(vehicle_tour3))
        print("Distance of vehicle tour 3: " + str(vehicle3_distance))
        total_dist = vehicle3_distance + vehicle1_distance + vehicle2_distance
        print("Total Distance of all vehicles: " + str(total_dist))
m._y = y
m.params.LazyConstraints = 1
m.optimize(subtourelim)

#m.optimize()
print_sol()

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 382 rows, 948 columns and 6711 nonzeros
Model fingerprint: 0xa46ee23b
Variable types: 3 continuous, 945 integer (945 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 103 rows and 163 columns
Presolve time: 0.08s
Presolved: 279 rows, 785 columns, 5045 nonzeros
Variable types: 3 continuous, 782 integer (780 binary)

Root relaxation: objective 3.625000e+03, 140 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4190.00000    0   21          - 4190.00000      -     -    0s
     0     

### Επεξήγηση Κώδικα - Μοντελοποίηση Προβλήματος Ερωτήματος 1
Στον παραπάνω κώδικα αφού ορίστηκαν τα δεδομένα από την εκφώνηση μας δημιουργήθηκαν δύο βοηθητικές συναρτήσεις subtourelim(model, where) και την subtour(adjList) για να μπορέσω να βρω και να δημιουργήσω lazy constraints για το subtour elimination. Στη συνέχεια, κάνω import τις βιβλιοθήκες του Gurobi και εισάγω τα δεδομένα σε αυτό για να αρχίσω να δημιουργώ το μοντέλο. Να σημειώσω ότι ο κώδικας δεν είναι τόσο δυναμικός αφού για την μοντελοποίηση του προβλήματος θεώρησα τις διαδρομές στατικές(δηλαδή τα φορτηγά) και έτσι μοντελοποιώντας το πρόβλημα ως vehicle routing problem παιρνόντας τον περιορισμό των 400 χιλιομέτρων του ζητούσα να μου κάνει minimize τις διαδρομές των φορτηγών που το έλεγα στην αρχή κάθε φορά. Έτσι, αφού δοκίμασα για ένα και ύστερα για 2 αυτοκίνητα ξεχωριστά παρατήρησα ότι το μοντέλο δεν έφτανε σε βέλτιστη λύση αντιθέτως ήταν αδύνατο. Άρα, ορίζοντας τα φορτηγά ως τρία το μοντέλο μου έβγαλε λύση που σημαίνει ο ελάχιστος αριθμός των φορτηγών είναι 3. Επίσης, ο κώδικας είναι στατικός για τα 3 φορτηγά και δεν παρουσιάζονται οι προσπάθειες που έγιναν με 1 ή με δύο φορτηγά. Επομένως, κατέληξα στη βέλτιστη λύση όπως τυπώθηκε και στην κονσόλα του κώδικα παραπάνω που παρουσιάζεται κι εδώ:
#### Διαδρομή Πρώτου Φορτηγού: Α -> Β9 -> Β8 - > Β1 -> Β12  Απόσταση: 240
#### Διαδρομή Δεύτερου Φορτηγού: Α -> Β3 -> Β13 - > Β6 -> Β14 -> Β7 -> Β2  Απόσταση: 350
#### Διαδρομή Τρίτου Φορτηγού: Α -> Β5 -> Β10 - > Β4 -> Β11: Απόσταση: 380
#### Συνολική Απόσταση: 970


### Ερώτημα 2 - Σχέδιο Σωτηρία 2
Στο δεύτερο ερώτημα εφόσον τα αυτοκίνητα που περισσεύουν ταξιδεύουν υπό τη μορφή κομβόι και μας ζητείται να ελαχιστοποιήσουμε το συνολικό χρόνο μεταφοράς ιατροφαρμακευτικού υλικού το πρόβλημα μας ανάγεται στο κλασσικό TSP όπου τα κόστη μας είναι οι χρόνοι. 

Παρακάτω παρουσιάζεται ο κώδικας για το δεύτερο ερώτημα:

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
import random
from itertools import combinations
import gurobipy as gp
from gurobipy import GRB

times = np.array([0, 786, 549, 657, 331, 559, 250, 
                 786, 0, 668, 979, 593, 224, 905,
                 549, 668, 0, 316, 607, 472, 467,
                 657, 979, 316, 0, 890, 769, 400,
                 331, 593, 607, 890, 0, 386, 559,
                 559, 224, 472, 469, 386, 0, 681,
                 250, 905, 467, 400, 559, 681, 0])

times = times.reshape((7, 7))


# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                                if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n:
            # add subtour elimination constr. for every pair of cities in tour
            model.cbLazy(gp.quicksum(model._vars[i, j]
                                     for i, j in combinations(tour, 2))
                         <= len(tour)-1)


# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = list(range(n))
    cycle = range(n+1)  # initial length has 1 more city
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle


# 
# Create n random points
n = 7

random.seed(1)
points = [(random.randint(0, 100), random.randint(0, 100)) for i in range(n)]

# Dictionary of Euclidean distance between each pair of points

dist = {(i,j):times[i][j] for i in range(7) for j in range(i)}
#print(dist)
m = gp.Model()

# Create variables

vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # edge in opposite direction

# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding m.addVars() call...
#
# vars = tupledict()
# for i,j in dist.keys():
#   vars[i,j] = m.addVar(obj=dist[i,j], vtype=GRB.BINARY,
#                        name='e[%d,%d]'%(i,j))


# Add degree-2 constraint

m.addConstrs(vars.sum(i, '*') == 2 for i in range(n))

# Using Python looping constructs, the preceding would be...
#
# for i in range(n):
#   m.addConstr(sum(vars[i,j] for j in range(n)) == 2)


# Optimize model

m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

print('')
print('Optimal tour: %s' % str(tour))
print('Optimal cost: %g' % m.objVal)
print('')

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 7 rows, 21 columns and 42 nonzeros
Model fingerprint: 0x91cd3dba
Variable types: 0 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 7 rows, 21 columns, 42 nonzeros
Variable types: 0 continuous, 21 integer (21 binary)
Found heuristic solution: objective 2650.0000000

Root relaxation: objective 2.575000e+03, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2575.0000000 2575.00000  0.00%     -    0s

Explored 0 nodes (

### Επεξήγηση Κώδικα - Μοντελοποίηση Ερωτήματος 2
Στο δεύτερο ερώτημα η υλοποίηση του κώδικα είναι παρόμοια με του πρώτου ερωτήματος και θα λέγαμε πιο απλή. Έτσι, κατασκευάζονται πάλι δυο ανάλογες συναρτήσεις για το subtour elimination lazy constraint. Στη συνέχεια, δημιουργούμε το μοντέλο από το gurobi και εισάγουμε τα δεδομένα μας και το μοντελοποιούμε με τα κατάλληλα constraints για πρόβλημα tsp. Δεν θα επεκταθούμε πολύ στην εξήγηση του κώδικα του δεύτερου ρωτήματος καθώς υπάρχει από την Gurobi για το πρόβλημα TSP ως παράδειγμα. Επομένως, ο κώδικας προσαρμόστηκε με βάση το παράδειγμα του TSP και το αποτέλεσμα μας είναι:
#### Βέλτιστη Διαδρομή του κομβόι: Α -> Γ4 -> Γ5 -> Γ1 -> Γ2 -> Γ3 -> Γ6 -> Α
#### Βέλτιστο Κόστος: 2575


### Πηγές - Βιβλιογραφία
Για την επίλυση του Project συμβουλεύτηκα το βιβλίο του Hillier και με βάση το παράδειγμα του TSP από την Gurobi λύθηκαν τα δύο προβλήματα.
Παράδειγμα ΤSP Gurobi: https://www.gurobi.com/documentation/9.1/examples/tsp_py.html?fbclid=IwAR1L9jGcfUWW3PwBgm56kE26f8vU7f3ATfwx4q-_y1CTEiDi8ooaeNZHycQ